In [1]:
import numpy as np
import pandas as pd

In [3]:
f = open('webkb-test-stemmed.txt')
lines = f.readlines()

In [4]:
bags = []
student = []
faculty = []
course = []
project = []
for line in lines:
  words = line.rstrip().split('\t')
  category = words[0]
  if len(words)>1:
    words = words[1].split(' ')
  else:
#     word = []
    print(category,'is empty')
    continue
#   word.append(category)
  word = list( filter( lambda s: isinstance(s, str) and len(s)>=3, words) )
  bags.extend(word)
  bags = list(set(bags))
  
  if category == 'student':
    student.append(word)
  elif category == 'faculty':
    faculty.append(word)
  elif category == 'course':
    course.append(word)
  elif category == 'project':
    project.append(word)
  else:
    print(category)
    print(word)
#   break

student is empty
faculty is empty
project is empty
course is empty
course is empty
project is empty
student is empty
student is empty
student is empty
faculty is empty
course is empty
faculty is empty
course is empty


In [5]:
len(bags)

4749

In [8]:
lines

['student\teric homepag eric wei tsinghua physic fudan genet\n',
 'course\tcomput system perform evalu model new sept assign due oct postscript text sept mimic librari public mimic inform lectur mwf comput scienc devis softwar home page html user manual postscript print file imag half hour initi instruct text mimic softwar tutori html postscript onlin html html professor miron livni offic comput scienc hour tba phone mail miron wisc teach assist chee chan offic comput scienc hour phone mail wisc suggest comment send wisc\n',
 'student\thome page comput scienc grad student ucsd work master degre origin edmonton alberta canada california good undergrad harvei mudd colleg california research work san diego supercomput center march initi work sdsc vrml browser implement network support month implement sdsc vrml behavior system demonstr vrml behavior workshop octob supercomput vrml decemb year work master thesi java applet interact scientif visual web find project implement sdsc vrml behavi

In [6]:
print(len(student))
print(len(faculty))
print(len(course))
print(len(project))


540
371
306
166


In [7]:
# bags to dictionary:
num_term = len(bags)
dic = {}
for i in range(num_term):
  dic[ bags[i] ] = i
# dic

# F

In [8]:
# student
num_student = len(student)
f_student = pd.DataFrame(np.zeros((num_student, num_term)), columns=bags)
for i in range(num_student):
  page = student[i]
  for word in page:
    f_student.loc[i, word] = f_student.loc[i, word] + 1
# faculty
num_faculty = len(faculty)
f_faculty = pd.DataFrame(np.zeros((num_faculty, num_term)), columns=bags)
for i in range(num_faculty):
  page = faculty[i]
  for word in page:
    f_faculty.loc[i, word] = f_faculty.loc[i, word] + 1
# course
num_course = len(course)
f_course = pd.DataFrame(np.zeros((num_course, num_term)), columns=bags)
for i in range(num_course):
  page = course[i]
  for word in page:
    f_course.loc[i, word] = f_course.loc[i, word] + 1
# project
num_project = len(project)
f_project = pd.DataFrame(np.zeros((num_project, num_term)), columns=bags)
for i in range(num_project):
  page = project[i]
  for word in page:
    f_project.loc[i, word] = f_project.loc[i, word] + 1
f_combine = pd.concat([f_student, f_faculty, f_course, f_project])
f_combine.shape

(1383, 4749)

In [9]:
f_student.to_csv('/home/share/qyq/GGM_real/WebKB/preprocess/f_student.csv', index=False)
f_faculty.to_csv('/home/share/qyq/GGM_real/WebKB/preprocess/f_faculty.csv', index=False)
f_course.to_csv('/home/share/qyq/GGM_real/WebKB/preprocess/f_course.csv', index=False)
f_project.to_csv('/home/share/qyq/GGM_real/WebKB/preprocess/f_project.csv', index=False)
f_combine.to_csv('/home/share/qyq/GGM_real/WebKB/preprocess/f_combine.csv', index=False)

In [10]:
terms_student = f_student.columns[~(f_student==0).all(axis=0)]
print(len(terms_student))
terms_faculty = f_faculty.columns[~(f_faculty==0).all(axis=0)]
print(len(terms_faculty))
terms_course = f_course.columns[~(f_course==0).all(axis=0)]
print(len(terms_course))
terms_project = f_project.columns[~(f_project==0).all(axis=0)]
print(len(terms_project))
terms_common = list(set(terms_student).intersection(terms_faculty).intersection(terms_course).intersection(terms_project))
len(terms_common)

3934
3864
3292
3026


1826

# log local weight

In [11]:
log_combine = np.log( 1+f_combine )
log_student = np.log( 1+f_student )
log_faculty = np.log( 1+f_faculty )
log_course = np.log( 1+f_course )
log_project = np.log( 1+f_project )

# Entropy global weight

In [13]:
p_combine = f_combine / f_combine.sum(axis=0)
p_student = f_student / f_student.sum(axis=0)
p_faculty = f_faculty / f_faculty.sum(axis=0)
p_course = f_course / f_course.sum(axis=0)
p_project = f_project / f_project.sum(axis=0)

In [15]:
def cal_entropy(col):
  entropy = - sum(col*np.where(col>0, np.log(col), 0))
  return entropy
entropy = p_combine.apply(cal_entropy, axis=0)
entropy

neighbor     1.039721
variabl      2.976139
hum          1.098612
canni        1.464816
visit        4.775908
               ...   
sto          1.609438
autom        4.070643
primarili    3.204778
cube         1.560710
sensor       2.553004
Length: 4749, dtype: float64

In [18]:
top_entropy = entropy.sort_values(ascending = False).head(100).index.values
top_entropy

array(['scienc', 'comput', 'univers', 'depart', 'page', 'research',
       'interest', 'home', 'inform', 'student', 'work', 'offic', 'system',
       'phone', 'public', 'program', 'email', 'mail', 'fax', 'project',
       'engin', 'link', 'group', 'graduat', 'includ', 'time', 'updat',
       'web', 'gener', 'develop', 'address', 'paper', 'area', 'fall',
       'languag', 'professor', 'softwar', 'teach', 'current', 'design',
       'applic', 'base', 'modifi', 'contact', 'list', 'relat', 'recent',
       'class', 'assist', 'algorithm', 'hour', 'studi', 'model',
       'analysi', 'institut', 'technolog', 'laboratori', 'implement',
       'introduct', 'www', 'number', 'construct', 'year', 'faculti',
       'network', 'center', 'note', 'topic', 'process', 'distribut',
       'hall', 'instructor', 'lab', 'colleg', 'problem', 'cours',
       'member', 'person', 'perform', 'click', 'structur', 'data',
       'architectur', 'build', 'send', 'educ', 'associ', 'access', 'site',
       'spring', '

In [59]:
index = 0
for t in top_entropy:
  
  if t not in terms_common:
    print(t)
    break
  index += 1
index

instructor


71

In [60]:
top_entropy[71]

'instructor'

In [20]:
joint_terms = ["spring", "fall", "public", "home", "page", "web", "site", "fax",
               "year", "person", "list", "phone", "email", "select", "offic", "instructor",
               "address", "problem", "book", "link", "relat", "topic", "hour",
               "work", "graduat", "number", "access", "theori", "class", "professor",
               "faculti", "algorithm", "time", "analysi", "student", "includ", "base",
               "gener", "structur", "data", "program", "model", "interest", "associ",
               "teach", "inform", "contact", "construct", "languag", "design", "project",
               "softwar", "applic", "system", "process", "area", "research", "group", "member",
               "implement", "parallel", "comput", "recent",
               "engin", "commun", "architectur", "distribut","perform", "high", "paper",
               "current", "laboratori", "lab", "support", "network", "advanc", "technolog",
               "studi", "introduct", "scienc", "univers", "depart", "educ", "www", "center", "institut",
              "send", "mail", "note", "assist", "develop"]
len(joint_terms)

91

In [21]:
print(len(list(set(top_entropy).intersection(joint_terms))))

91


In [22]:
def log_entropy(col):
  log_entropy = 1 + sum(col*np.where(col>0, np.log(col), 0)) / np.log( len(col) )
  return log_entropy
e_combine = p_combine.apply(log_entropy, axis=0)
e_student = p_student.apply(log_entropy, axis=0)
e_faculty = p_faculty.apply(log_entropy, axis=0)
e_course = p_course.apply(log_entropy, axis=0)
e_project = p_project.apply(log_entropy, axis=0)

In [49]:
x_combine = log_combine * e_combine
x_student = log_student * e_student
x_faculty = log_faculty * e_faculty
x_course = log_course * e_course
x_project = log_project * e_project

In [33]:
x_combine_top = x_combine[top_entropy]
x_student_top = x_student[top_entropy]
x_faculty_top = x_faculty[top_entropy]
x_course_top = x_course[top_entropy]
x_project_top = x_project[top_entropy]

x_combine_top.to_csv('entropy_combine.csv', index=False)
x_student_top.to_csv('entropy_student.csv', index=False)
x_faculty_top.to_csv('entropy_faculty.csv', index=False)
x_course_top.to_csv('entropy_course.csv', index=False)
x_project_top.to_csv('entropy_project.csv', index=False)

In [34]:
x_combine_top_student = x_combine_top[:num_student]
x_combine_top_faculty = x_combine_top[num_student: num_student+num_faculty]
x_combine_top_course = x_combine_top[num_student+num_faculty: num_student+num_faculty+num_course]
x_combine_top_project = x_combine_top[num_student+num_faculty+num_course:]

x_combine_top_student.to_csv('entropy_combine_student.csv', index=False)
x_combine_top_faculty.to_csv('entropy_combine_faculty.csv', index=False)
x_combine_top_course.to_csv('entropy_combine_course.csv', index=False)
x_combine_top_project.to_csv('entropy_combine_project.csv', index=False)

In [30]:
num_student+num_faculty+num_course+num_project

1383

In [42]:
x_project_top['instructor']
# x_combine_top_project['instructor'].sum()

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
161   NaN
162   NaN
163   NaN
164   NaN
165   NaN
Name: instructor, Length: 166, dtype: float64

In [55]:
x_project_top = x_project_top.dropna(axis=1)
x_project_top
x_project_top.to_csv('entropy_project.csv', index=False)

In [57]:
x_combine_top_project = x_combine_top_project.drop(columns=['instructor'])
x_combine_top_project

,scienc,comput,univers,depart,page,research,interest,home,inform,student,...,parallel,theori,commun,high,resourc,book,select,support,specif,advanc
0,0.093404,0.093446,0.139733,0.000000,0.000000,0.180932,0.000000,0.000000,0.203955,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.223525,0.0,0.225673,0.000000,0.000000,0.000000
1,0.058931,0.058958,0.069866,0.081381,0.000000,0.233852,0.000000,0.000000,0.101977,0.000000,...,0.349751,0.000000,0.000000,0.222651,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,0.058931,0.058958,0.110736,0.081381,0.173401,0.143385,0.000000,0.156998,0.101977,0.181689,...,0.000000,0.000000,0.222205,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.225892
3,0.000000,0.000000,0.000000,0.000000,0.086700,0.000000,0.000000,0.000000,0.000000,0.114633,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.058958,0.000000,0.000000,0.000000,0.143385,0.093573,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,0.058931,0.136896,0.069866,0.000000,0.000000,0.143385,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.515945,0.445301,0.223525,0.0,0.000000,0.225734,0.000000,0.225892
162,0.058931,0.058958,0.110736,0.081381,0.137417,0.000000,0.000000,0.156998,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
163,0.152335,0.093446,0.069866,0.000000,0.000000,0.300522,0.093573,0.000000,0.286287,0.436449,...,0.000000,0.699772,0.704374,0.575544,0.000000,0.0,0.357683,0.000000,0.000000,0.000000
164,0.117862,0.093446,0.110736,0.081381,0.000000,0.286771,0.093573,0.000000,0.000000,0.000000,...,0.000000,0.220754,0.222205,0.000000,0.000000,0.0,0.000000,0.225734,0.225872,0.225892


In [58]:
x_combine_top_project.to_csv('entropy_combine_project.csv', index=False)

In [21]:
x_concate = pd.concat([x_student, x_faculty, x_course, x_project])
x_concate.sum(axis=0).sort_values(ascending = False).head(100).index.values

array(['system', 'program', 'comput', 'project', 'page', 'parallel',
       'inform', 'languag', 'design', 'research', 'softwar', 'engin',
       'work', 'scienc', 'network', 'time', 'univers', 'distribut',
       'algorithm', 'home', 'data', 'problem', 'model', 'offic', 'paper',
       'develop', 'perform', 'learn', 'group', 'oper', 'applic',
       'postscript', 'link', 'web', 'gener', 'process', 'depart',
       'includ', 'object', 'class', 'interest', 'architectur', 'student',
       'confer', 'mail', 'machin', 'technolog', 'implement', 'professor',
       'base', 'intellig', 'report', 'theori', 'analysi', 'visual',
       'databas', 'compil', 'graduat', 'email', 'commun', 'lab', 'list',
       'graphic', 'click', 'version', 'support', 'intern', 'public',
       'mathemat', 'document', 'phone', 'code', 'assign', 'proceed',
       'schedul', 'environ', 'laboratori', 'present', 'high', 'area',
       'technic', 'member', 'memori', 'file', 'imag', 'current', 'www',
       'test', 'rea

In [23]:
x_norm_student = x_student / np.linalg.norm(x_student, axis=0)
x_norm_faculty = x_faculty / np.linalg.norm(x_faculty, axis=0)
x_norm_course = x_course / np.linalg.norm(x_course, axis=0)
x_norm_project = x_project / np.linalg.norm(x_project, axis=0)
x_norm_concate = pd.concat([x_norm_student, x_norm_faculty, x_norm_course, x_norm_project])
x_norm_concate.sum(axis=0).sort_values(ascending = False).head(100).index.values

array(['comput', 'scienc', 'univers', 'page', 'student', 'research',
       'depart', 'inform', 'system', 'home', 'interest', 'project',
       'program', 'work', 'offic', 'engin', 'mail', 'group', 'time',
       'email', 'phone', 'includ', 'link', 'public', 'languag',
       'professor', 'design', 'develop', 'gener', 'softwar', 'paper',
       'graduat', 'applic', 'web', 'updat', 'class', 'fax', 'base',
       'algorithm', 'network', 'address', 'area', 'current', 'list',
       'model', 'problem', 'distribut', 'relat', 'process', 'data',
       'fall', 'modifi', 'parallel', 'contact', 'implement', 'perform',
       'teach', 'technolog', 'analysi', 'laboratori', 'studi', 'lab',
       'recent', 'topic', 'construct', 'www', 'oper', 'number',
       'architectur', 'assist', 'center', 'object', 'commun', 'theori',
       'member', 'support', 'click', 'structur', 'machin', 'high', 'note',
       'introduct', 'hour', 'build', 'year', 'colleg', 'hall', 'report',
       'institut', 'access', 

In [13]:
entropy = p_student
def cal_entropy(col):
  entropy = - sum(col*np.where(col>0, np.log(col), 0))
  return entropy
entropy = entropy.apply(cal_entropy, axis=0)
entropy

particl    0.693147
reli      -0.000000
conwai          NaN
comment    3.629374
taho      -0.000000
             ...   
neuron     1.475076
desir     -0.000000
tap             NaN
hoar            NaN
hr        -0.000000
Length: 4800, dtype: float64

In [14]:
top_hubs = entropy.sort_values(ascending = False).head(100).index.values
top_hubs

array(['student', 'scienc', 'comput', 'univers', 'depart', 'page', 'home',
       'interest', 'research', 'work', 'mail', 'graduat', 'offic',
       'engin', 'phone', 'inform', 'link', 'system', 'web', 'email',
       'public', 'time', 'project', 'program', 'resum', 'address', 'fax',
       'group', 'updat', 'contact', 'person', 'advisor', 'year', 'site',
       'modifi', 'construct', 'network', 'institut', 'develop', 'languag',
       'area', 'click', 'paper', 'current', 'list', 'softwar',
       'technolog', 'stuff', 'includ', 'internet', 'www', 'usa', 'visit',
       'distribut', 'search', 'lab', 'master', 'homepag', 'design',
       'educ', 'world', 'thesi', 'degre', 'pictur', 'phd', 'check',
       'gener', 'state', 'number', 'hall', 'studi', 'find', 'send',
       'class', 'favorit', 'school', 'info', 'recent', 'california',
       'model', 'place', 'center', 'colleg', 'parallel', 'databas',
       'fall', 'algorithm', 'good', 'html', 'music', 'assist', 'dept',
       'base', 'fi

In [16]:
x_student_hubs = e_student[top_hubs] * np.log( 1+f_student[top_hubs] )
x_student_hubs

,student,scienc,comput,univers,depart,page,home,interest,research,work,...,assist,dept,base,finger,report,read,teach,relat,perform,access
0,0.010938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.017336,0.083912,0.054636,0.000000,0.000000,0.076445,0.082026,0.144424,0.093628,0.270710,...,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.017336,0.105886,0.126861,0.092394,0.000000,0.121162,0.130009,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.266465,0.00000,0.000000,0.000000,0.000000,0.000000,0.270304
3,0.030706,0.105886,0.181497,0.092394,0.116966,0.242324,0.246079,0.182243,0.093628,0.184788,...,0.00000,0.0,0.265567,0.000000,0.00000,0.535351,0.000000,0.269233,0.000000,0.000000
4,0.017336,0.083912,0.126861,0.092394,0.000000,0.121162,0.164053,0.000000,0.093628,0.184788,...,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,0.010938,0.000000,0.000000,0.000000,0.000000,0.076445,0.130009,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
540,0.010938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
541,0.017336,0.052943,0.109272,0.116588,0.000000,0.000000,0.000000,0.000000,0.148397,0.116588,...,0.00000,0.0,0.420914,0.000000,0.00000,0.000000,0.000000,0.000000,0.427595,0.000000
542,0.021875,0.105886,0.153382,0.058294,0.000000,0.076445,0.000000,0.182243,0.217397,0.270710,...,0.26474,0.0,0.531135,0.000000,0.26754,0.000000,0.425870,0.000000,0.000000,0.428422


In [17]:
x_student_hubs.to_csv('hubs_student.csv', index=False)